In [ ]:
############################################################## START ##############################################################

In [ ]:
!wget http://www.statmt.org/europarl/v7/europarl.tgz
!tar zxvf europarl.tgz
# !ls

In [ ]:
import os
import re
import ast
import copy
import time
import tqdm
import torch
import numpy as np
import pandas as pd
import pickle as pkl
import torch.nn as nn
import torch.functional as F
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [18]:
SEED=44
EPOCHS=5
DEVICE="gpu" 
DROPOUT=0.2
BATCH_SIZE=16
L2_DECAY=0.001
LEARNING_RATE=1e-4
CHAR_VECTOR_DIM=100
# CLIP=False
if DEVICE == "gpu":
    DEVICE=("cuda:0" if torch.cuda.is_available() else "cpu")

In [19]:
### ALL CLASSES: ['bg', 'cs', 'da', 'de', 'el', 'en', 'es', 'et', 'fi', 'fr', 'hu', 'it', 'lt', 'lv', 'nl', 'pl', 'pt', 'ro', 'sk', 'sl', 'sv']
LABELS = ['en', 'fr', 'it', 'pt', 'nl', 'es', 'da', 'de']
LABEL_MAP = {}

for i in range(len(LABELS)):
    # LABEL_MAP[LABELS[i]] = torch.zeros(len(LABELS))
    # LABEL_MAP[LABELS[i]][i] = 1
    LABEL_MAP[LABELS[i]] = torch.as_tensor([i]) 
LABEL_MAP

{'da': tensor([6]),
 'de': tensor([7]),
 'en': tensor([0]),
 'es': tensor([5]),
 'fr': tensor([1]),
 'it': tensor([2]),
 'nl': tensor([4]),
 'pt': tensor([3])}

In [20]:
char_set = {'a'}

In [24]:
def process_dataset(path='txt', op_path='dataset', limit=10000, filter=[]):
    LONGEST_WORD = (0, '')
    inputs = []
    targets = []
    try:
        os.mkdir(op_path)
    except FileExistsError:
        pass
    ind = 0
    for dir in filter:
        lang = dir
        ctr, k = 0, 0
        print(lang)
        for txt in tqdm.tqdm(os.listdir(path+'/'+dir), position=0, leave=True):
            try:
                df = []
                ind += 1
                for line in open(path+'/'+dir+'/'+txt, "r"):
                    # ctr += 1
                    for word in line.split():
                        for punct in ['/', '\\', '_']:
                            if punct in word:
                                line = line.replace(word, "")
                    line = re.sub("<.*>", "", line)
                    line = re.sub(r'http\S+', '', line)
                    for punct in ['\t','\n','!','"','#','$','%','&',"'",')','(','*','+',',','-','.','/',':',';','<','=','>','?','@','^','//']:
                        line = line.replace(punct, " ")
                    for punct in ['.']:
                        line = line.replace(punct, " ")
                    line = ' '.join(line.split())

                    for letter in line:
                        char_set.add(letter)
                    if line != '':
                        k += 1
                        for word in line.split():
                            if len(word) > LONGEST_WORD[0]:
                                LONGEST_WORD = (len(word), word)
                        inputs.append(line)
                        targets.append(lang)
                        if k == limit:
                            break
                    if k == limit:
                        break
                # df.to_csv(op_path+f"/{ind}.csv")
            except UnicodeDecodeError:
                print(path+'/'+dir+'/'+txt)
        # data_dist[lang] = ctr
    return LONGEST_WORD, inputs, targets
filter = ['en', 'fr', 'it', 'pt', 'nl', 'es', 'da', 'de']
LONGEST_WORD, inputs, targets = process_dataset(filter=filter)

  0%|          | 19/9672 [00:00<01:32, 104.86it/s]

en


  0%|          | 0/9450 [00:00<?, ?it/s]

fr


  0%|          | 18/9486 [00:00<01:37, 97.42it/s]

...

  0%|          | 18/9373 [00:00<01:31, 101.69it/s]

da


  0%|          | 16/9224 [00:00<01:49, 83.72it/s]

de


100%|██████████| 9224/9224 [00:01<00:00, 7704.28it/s]


In [ ]:
# data_dist = {}
# for lang in LABEL_MAP:
#     data_dist[lang]=0  

In [ ]:
# for k,v in sorted(data_dist.items(), key=lambda x: x[1], reverse=True):
#     print(k, v)

### Sentences per language
"""
en 1663354
fr 1637529
it 1626908
pt 1616234
nl 1615893
es 1611610
da 1603101
de 1567229
sv 1562275
fi 1536972
el 1193948
et 476752
cs 476020
sk 470712
lt 470624
lv 469128
pl 467987
hu 461872
sl 458136
bg 287826
ro 283680
"""

In [22]:
char_list = sorted(list(char_set))
char_map = {}
for i, char in enumerate(char_list):
    char_map[char]=i
len(char_map)
# char_map

261

In [23]:
LONGEST_WORD

(45, 'infrastrukturmoderniseringsforanstaltningerne')

In [25]:
class CharTokenizer():
    def __init__(self, char_map, label_map=LABEL_MAP):
        self.map = char_map
        self.label_map = label_map

    def encode(self, sentence, pad=47):
        res = []
        for word in sentence.split():
            temp = []
            for letter in word:
                temp.append(self.map[letter])
            while len(temp)<pad:
                temp.append(0)
            temp = torch.as_tensor(temp)
            res.append(temp)
        res = torch.stack(res)

        return res

    def encode_target(self, label):
        return self.label_map[label]

    def encode_dataset(self, sentences, targets):
        res = []
        tgts = []
        for i in range(len(sentences)):
            res.append(self.encode(sentences[i]))
            tgts.append(self.encode_target(targets[i]))
        # res = torch.stack(res)
        # targets = torch.stack(targets) 
        tgts = torch.cat(tgts)
        return res, tgts
tokenizer = CharTokenizer(char_map=char_map, label_map=LABEL_MAP)

In [ ]:
# !find "batches" -type f | wc -l
# !rm -rf -d "dataset"

In [ ]:
# def tokenize_dataset(ip_path='dataset', op_path='processed', batch_size=BATCH_SIZE):
#     try:
#         os.mkdir(op_path)
#     except FileExistsError:
#         pass
    
#     proc, target = [], []
#     for item in os.listdir(ip_path):
#         df = pd.read_csv(ip_path+'/'+item)
#         proc += list(df['text'])
#         for label in list(df['label']):
#             target.append(LABEL_MAP[label])
#         # for item in df.to_dict('record'):
#         #      = tokenizer.encode(item['text'])
#         #     LABEL_MAP[item['label']]
#     proc = tokenizer(proc)
#     target = torch.stack(target)

#     return proc, target

# proc, target = tokenize_dataset()

In [26]:
class CharLSTM(nn.Module):
    def __init__(self, char_set_size=261, hidden_size=50, word_pad=47, num_classes=8):
        super(CharLSTM, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=char_set_size, embedding_dim=CHAR_VECTOR_DIM)
        self.lstm = nn.LSTM(input_size=CHAR_VECTOR_DIM, hidden_size=hidden_size)
        self.dropout = nn.Dropout(p=DROPOUT)
        self.max_pooler = nn.MaxPool1d(kernel_size=word_pad)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, sentences):
        y = []
        # print(f"sentences are: {sentences}")
        for x in sentences:
            x.to(DEVICE)
            x = self.embedding(x)
            x,_ = self.lstm(x)
            x = torch.transpose(x[-1:], 1, 2)
            x = self.max_pooler(x)
            y.append(x[0].squeeze(1))

        y = torch.stack(y)
        y = self.fc(y)
        y = self.softmax(y)

        return y

In [27]:
model = CharLSTM()
model.to(DEVICE)
print(model)

CharLSTM(
  (embedding): Embedding(261, 100)
  (lstm): LSTM(100, 50)
  (dropout): Dropout(p=0.2, inplace=False)
  (max_pooler): MaxPool1d(kernel_size=47, stride=47, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=50, out_features=8, bias=True)
  (softmax): Softmax(dim=1)
)


In [28]:
# target_inputs, val_inputs
criterion = nn.CrossEntropyLoss()
x_train, x_test_val, y_train, y_test_val = train_test_split(inputs, targets, test_size=0.2, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_test_val, y_test_val, test_size=0.5, random_state=42)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [29]:
training_stats = []
for epoch in range(EPOCHS):
    print('\n======== Epoch {:} / {:} ========\n'.format(epoch+1, EPOCHS))
    model.train()
    train_batch_losses = []
    ITERS = int(len(x_train)/BATCH_SIZE)
    for i in tqdm.tqdm(range(ITERS), position=0, leave=True):
        end = BATCH_SIZE*(i+1)
        start = BATCH_SIZE*i
        model.zero_grad()
        input, target = tokenizer.encode_dataset(x_train[start : end], y_train[start : end])
        
        for i in range(len(input)):
            input[i] = input[i].to(DEVICE)

        output = model(input)
        output = output.to(DEVICE)
        target = target.to(DEVICE)
        loss = criterion(output, target)
        # print(loss.item())
        # data.append({"output":output.tolist(), "target":target.tolist()})
        loss.backward() 
        optim.step()
        train_batch_losses.append(loss)

    val_batch_losses = []
    model.eval()
    ITERS = int(len(x_val)/BATCH_SIZE)
    for i in tqdm.tqdm(range(ITERS), position=0, leave=True):
        end = BATCH_SIZE*(i+1)
        start = BATCH_SIZE*i
        input, target = tokenizer.encode_dataset(x_val[start : end], y_val[start : end])

        for i in range(len(input)):
            input[i] = input[i].to(DEVICE)
        
        with torch.no_grad():
            output = model(input)
            output = output.to(DEVICE)
            target = target.to(DEVICE)
            loss = criterion(output, target)
        # print(loss.item())
        # data.append({"output":output.tolist(), "target":target.tolist()})
            val_batch_losses.append(loss)
    # for step, batch in enumerate(tqdm.tqdm(val_dataloader)):
    #     with torch.no_grad():
    #         output = model(input_ids, attn_masks, token_ids)
    #         output.to(DEVICE)
    #         loss = criterion(output, target)
    #             # print(loss.item())
    #         val_batch_losses.append(loss)
                # val_stats['mse_loss'].append(get_mse(output, target))
                # val_stats['r'].append(get_r(output, target))
                # val_stats['kl_div_loss'].append(get_kl(output, target))
    
    train_dict = {
                'Epoch': epoch + 1,
                'Training Loss': torch.tensor(train_batch_losses).mean().item(),
                'Validation Loss': torch.tensor(val_batch_losses).mean().item(),
                }
   
    # curr_val_loss = train_dict['Validation Loss']
    # curr_train_loss = train_dict['Training Loss']
    # if curr_val_loss < min_val_loss:
    #     min_val_loss = curr_val_loss
    #     print("Saving model ...")
    #     torch.save(model, "best_model")

    # if curr_train_loss < min_train_loss:
    #     min_train_loss = curr_train_loss
        
    #     if curr_val_loss == min_val_loss:
    #         print("Saving model ...")
    #         torch.save(model, "best_model")
    
    print(str(train_dict)+'\n')
    training_stats.append(train_dict)

print("Training complete!")

  0%|          | 1/4000 [00:00<08:10,  8.16it/s]


======== Epoch 1 / 5 ========



100%|██████████| 500/500 [00:19<00:00, 25.51it/s]


{'Epoch': 1, 'Training Loss': 1.80635404586792, 'Validation Loss': 1.5256998538970947}


======== Epoch 2 / 5 ========



100%|██████████| 500/500 [00:19<00:00, 25.54it/s]


...

{'Epoch': 3, 'Training Loss': 1.380031943321228, 'Validation Loss': 1.3585914373397827}


======== Epoch 4 / 5 ========



100%|██████████| 500/500 [00:18<00:00, 27.15it/s]


{'Epoch': 4, 'Training Loss': 1.3461110591888428, 'Validation Loss': 1.339268684387207}


======== Epoch 5 / 5 ========



100%|██████████| 500/500 [00:18<00:00, 27.34it/s]


{'Epoch': 5, 'Training Loss': 1.331242561340332, 'Validation Loss': 1.3295758962631226}

Training complete!


In [30]:
def predict(model, inputs, tokenizer):
    model.eval()
    model.to(DEVICE)
    outputs = []
    
    invert_map = {}
    map = tokenizer.label_map
    for key in map:
        value = map[key].tolist()[0]
        invert_map[value] = key

    for input in tqdm.tqdm(inputs, position=0, leave=True):
        input = tokenizer.encode(input)
        with torch.no_grad():
            input = input.to(DEVICE)
            output = model([input])
            output = output.to('cpu')
            outputs.append(invert_map[int(np.argmax(output))])
        # print(loss.item())
        # data.append({"output":output.tolist(), "target":target.tolist()})
    return outputs

preds = predict(model, x_test, tokenizer)

100%|██████████| 8000/8000 [00:22<00:00, 360.17it/s]


In [31]:
def get_accuracy(preds, true, norm=True):
    score=0
    for x,y in zip(preds, true):
        if x == y:
            score += 1
    if norm:
        score /= len(true)
    return score

iso_map = {'nl':'Dutch', 'es':'Spanish', 'it':'Italian', 'fr':'French', 'pt':'Portugese', 'en':'English', 'de':'German', 'da':'Danish'}
test_dist = {iso_map[class_]:y_test.count(class_) for class_ in set(y_test)}

def get_class_wise_accuracy(preds, true, norm=True):
    classes = set(true+preds)
    accuracy = {class_:0 for class_ in classes}
    totals = {class_:0 for class_ in classes}
    
    for x,y in zip(preds, true):
        if x == y:
            accuracy[x] += 1
        totals[x] += 1
    if norm:
        for class_ in accuracy:
            accuracy[class_] /= totals[class_] 
    accuracy = {iso_map[class_]:accuracy[class_] for class_ in accuracy}
    return accuracy

# print(f"{y_test}")

print(f"test set class distribution {test_dist}")
print(f"accuracy = {get_accuracy(preds, y_test)*100}%")
print("class wise accuracies are:")
accuracy = get_class_wise_accuracy(preds, y_test)
for class_ in accuracy:
    print(f"{class_}: accuracy = {accuracy[class_]*100}%")

print(f"f1-micro is {f1_score(preds, y_test, average='micro')*100}%")
print(f"f1-macro is {f1_score(preds, y_test, average='macro')*100}%")
print(f"f1-weighted is {f1_score(preds, y_test, average='weighted')*100}%")

test set class distribution {'Italian': 981, 'English': 965, 'German': 1048, 'Dutch': 963, 'Portugese': 1022, 'French': 974, 'Spanish': 1020, 'Danish': 1027}
accuracy = 95.525%
class wise accuracies are:
Italian: accuracy = 93.68213228035538%
English: accuracy = 97.89251844046365%
German: accuracy = 87.82755705832629%
Dutch: accuracy = 98.59002169197397%
Portugese: accuracy = 96.31474103585657%
French: accuracy = 97.53483386923901%
Spanish: accuracy = 96.91040164778579%
Danish: accuracy = 97.36585365853658%
f1-micro is 95.525%
f1-macro is 95.56758391674758%
f1-weighted is 95.50344766088578%


In [32]:
lengths = []
mismatches = []
correct_lengths = []
mistake_lengths = []

for sent, true, pred in zip(x_test, y_test, preds):
    if true != pred:
        mismatches.append({'text':sent, 'true label':true, 'prediction':pred})
        mistake_lengths.append(len(sent.strip().split()))
    else:
        correct_lengths.append(len(sent.strip().split()))
    lengths.append(len(sent.strip().split()))
mismatches = pd.DataFrame(mismatches)
print("misclassifications:")
mismatches

misclassifications:


,text,true label,prediction
0,Sobre el apartado 5,es,pt
1,Turno de preguntas preguntas al Consejo,es,it
2,Murder of Rosemary Nelson,en,da
3,Excluir a determinados usuarios de este sistem...,es,fr
4,Muchas gracias señor Söderman,es,de
...,...,...,...
353,Madame la Commissaire faites preuve de plus de...,fr,it
354,La ringrazio Commissario Van den Broek,it,es
355,A Uachtaráin a Leas Uachtaráin ní mór don Choi...,de,en
356,SPEAKER ID 212 NAME Nils Lundgren,it,de


In [33]:
# analysis of sentence lengths
mean = np.mean(lengths)
median = np.median(lengths)
std_dev = (np.var(lengths))**0.5
print(f"the mean of length of sentences = {mean}")
print(f"the median of length of sentences = {median}")
print(f"the standard deviation of length of sentences = {std_dev}")

the mean of length of sentences = 70.706375
the median of length of sentences = 63.0
the standard deviation of length of sentences = 47.60522985302534


In [34]:
# analysis of sentence lengths for wrong predictions
mean = np.mean(mistake_lengths)
median = np.median(mistake_lengths)
std_dev = (np.var(mistake_lengths))**0.5
print(f"the mean of length of misclassified sentences = {mean}")
print(f"the median of length of misclassified sentences = {median}")
print(f"the standard deviation of length of misclassified sentences = {std_dev}")

the mean of length of misclassified sentences = 10.986033519553073
the median of length of misclassified sentences = 6.0
the standard deviation of length of misclassified sentences = 15.801043930543507


In [35]:
# analysis of sentence lengths for correct predictions
mean = np.mean(correct_lengths)
median = np.median(correct_lengths)
std_dev = (np.var(correct_lengths))**0.5
print(f"the mean of length of correcty classified sentences = {mean}")
print(f"the median of length of correctly classified sentences = {median}")
print(f"the standard deviation of length of correctly classified sentences = {std_dev}")

the mean of length of correcty classified sentences = 73.50405652970427
the median of length of correctly classified sentences = 65.0
the standard deviation of length of correctly classified sentences = 46.75277622867615


In [ ]:
### These are details for model performance on first 10,000 sentences of each language
### train : validation : test = 80% : 10% : 10%  
"""
test set class distribution {'Spanish': 1020, 'Italian': 981, 'French': 974, 'Portugese': 1022, 'Dutch': 963, 'English': 965, 'German': 1048, 'Danish': 1027}
accuracy = 95.875%
class wise accuracies are:
Spanish: accuracy = 96.07250755287009%
Italian: accuracy = 93.78698224852072%
French: accuracy = 97.00103412616339%
Portugese: accuracy = 96.79037111334003%
Dutch: accuracy = 98.69989165763813%
English: accuracy = 97.97872340425532%
German: accuracy = 90.04366812227073%
Danish: accuracy = 97.84524975514202%
f1-micro is 95.875%
f1-macro is 95.90623350666452%
f1-weighted is 95.86089203953024%
"""
### Results of sentence length analysis
"""
the mean of length of sentences = 70.706375
the median of length of sentences = 63.0
the standard deviation of length of sentences = 47.60522985302534

the mean of length of misclassified sentences = 9.912121212121212
the median of length of misclassified sentences = 6.0
the standard deviation of length of misclassified sentences = 15.681918898812862

the mean of length of correcty classified sentences = 73.32203389830508
the median of length of correctly classified sentences = 65.0
the standard deviation of length of correctly classified sentences = 46.76883228784532
"""

In [ ]:
### These are details for model performance on first 10,000 sentences of each language
### train : validation : test = 80% : 10% : 10%  
"""
test set class distribution {'Spanish': 12575, 'German': 12523, 'Italian': 12434, 'French': 12722, 'Portugese': 12500, 'English': 12280, 'Dutch': 12474, 'Danish': 12492}
accuracy = 98.28%
class wise accuracies are:
Spanish: accuracy = 98.79644588045234%
German: accuracy = 98.16681215776526%
Italian: accuracy = 97.92299089311392%
French: accuracy = 98.67685279987398%
Portugese: accuracy = 96.94271531006224%
English: accuracy = 97.97659678205754%
Dutch: accuracy = 98.36237495966441%
Danish: accuracy = 99.4198694706309%
f1-micro is 98.28%
f1-macro is 98.27959967758044%
f1-weighted is 98.2794312965243%
"""

In [ ]:
############################################################## END ##############################################################